In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

In [0]:
!pip install -q requests

import numpy as np
import requests
import json
import os

In [3]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   9909      0 --:--:-- --:--:-- --:--:--  9909
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.

In [0]:
os.environ["MODEL_DIR"] = '/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/model/gru_seq2seq_export'

In [5]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8508 \
  --model_name=free_chat_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [6]:
!tail server.log

2019-10-11 00:33:16.940878: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2019-10-11 00:33:16.941008: I tensorflow_serving/model_servers/server_core.cc:561]  (Re-)adding model: free_chat_model
2019-10-11 00:33:17.369398: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: free_chat_model version: 1569460583}
2019-10-11 00:33:17.369583: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: free_chat_model version: 1569460583}
2019-10-11 00:33:17.369704: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: free_chat_model version: 1569460583}
2019-10-11 00:33:17.370153: I external/org_tensorflow/tensorflow/contrib/session_bundle/bundle_shim.cc:363] Attempting to load native SavedModelBundle in bundle-shim from: /content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/model/gru_seq2seq_export/1569460583
2019-10-11 00:33:17.370279: I exter

In [0]:
def get_vocab(f_path):
  k2v = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      k2v[line] = i
  return k2v

parse_fn = lambda text: [[CHAR2IDX.get(c, len(CHAR2IDX)) for c in list(text)]]

In [8]:
CHAR2IDX = get_vocab('../vocab/char.txt')
data = json.dumps({"signature_name": "serving_default", "instances": parse_fn('你是谁')})
print(data)

{"signature_name": "serving_default", "instances": [[10, 13, 119]]}


In [9]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8508/v1/models/free_chat_model:predict', data=data, headers=headers)
print(json_response)
predictions = json.loads(json_response.text)['predictions']

<Response [200]>


In [10]:
predictions = np.asarray(predictions)
IDX2CHAR = {idx: char for char, idx in CHAR2IDX.items()}
for j in range(5):
  print('A{}:'.format(j+1), ' '.join([IDX2CHAR.get(idx, len(IDX2CHAR)) for idx in predictions[0, :, j]]).replace('<end>', ''))

A1: 我 是 小 通                  
A2: 我 是 小 黄 鹰                 
A3: 不 是 她 们 俩 吗 ？               
A4: 我 是 小 通 ~                 
A5: 我 是 宇 宙 无 敌 可 爱 小 通            
